In [47]:
import numpy as np
import pandas as pd

import tensorflow as tf

import tensorflow_hub as hub

import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
df = pd.read_csv('train.csv')

In [49]:
texts = []
for i in range(len(df['Filename'])):
    fn = df['Filename'][i]
    folder = fn.split("-")[0]
    f = open('train-data/'+folder+'/'+fn, "rb")
    texts.append(f.read())
df['text'] = texts

In [65]:
intervention = df.copy().drop(['Blinding of Outcome assessment','Classes'],axis=1)

In [51]:
intervention

,Filename,Blinding of intervention,text
0,00060-02.txt,P,b'A Multicomponent Intervention To Prevent Maj...
1,00060-03.txt,N,b'Original article\r\nSelf-management versus c...
2,00060-04.txt,N,b'Original Research\r\nHome Management of Oral...
3,00060-05.txt,P,b'For personal use only. Not to be reproduced ...
4,00060-06.txt,P,b'\xc2\xa9 2016 Pozzi et al. This work is publ...
...,...,...,...
623,00458-20.txt,N,"b""\xef\xbb\xbfEffect of daily versus twice wee..."
624,00458-21.txt,P,b'\xef\xbb\xbfEur J Nutr 39 : 263\xe2\x80\x932...
625,00458-22.txt,N,"b""\xef\xbb\xbfARTICLE\r\n \r\n\r\n \r\nWeekly ..."
626,00458-24.txt,N,b'\xef\xbb\xbfSymposium: Improving Adolescent ...


In [132]:
train_labels,train_examples = intervention['Blinding of intervention'],intervention['text'].to_numpy()
train_labels = train_labels.replace(['N','P','Q'],[0,1,2]).to_numpy()

In [133]:
print("Training entries: {}".format(len(train_examples)))

Training entries: 628


In [134]:
train_examples[:10]

array([b'A Multicomponent Intervention To Prevent Major Bleeding\r\nComplications in Older Patients Receiving Warfarin\r\nA Randomized, Controlled Trial\r\nRebecca J. Beyth, MD, MS; Linda Quinn, MS; and C. Seth Landefeld, MD\r\nBackground: Warfarin is effective in the treatment and prevention of many venous thromboembolic disorders, but it often leads\r\nto bleeding.\r\nObjective: To develop a multicomponent program of management of warfarin therapy and to determine its effect on the frequency of warfarin-related major bleeding in older patients.\r\nDesign: Randomized, controlled trial.\r\nSetting: University hospital in Cleveland, Ohio.\r\nPatients: 325 patients 65 years of age or older who started\r\nwarfarin therapy during hospitalization.\r\nInterventions: Patients were stratified according to baseline risk\r\nfor major bleeding and were randomly assigned to receive the\r\nintervention (n 5 163) or usual care (n 5 162) by their primary\r\nphysicians for 6 months. The intervention c

In [135]:
train_labels[:10]

array([1, 0, 0, 1, 1, 0, 1, 0, 1, 0], dtype=int64)

In [140]:
model = ("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1")

hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Embedding(6000, 16))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_15 (KerasLayer)  (None, 20)                400020    
_________________________________________________________________
embedding_8 (Embedding)      (None, 20, 16)            96000     
_________________________________________________________________
global_average_pooling1d_3 ( (None, 16)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 17        
Total params: 496,309
Trainable params: 496,309
Non-trainable params: 0
_________________________________________________________________


In [141]:
model.compile(
    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.optimizers.Adam(), 
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")])

In [142]:
x_val = train_examples[:100]
partial_x_train = train_examples[100:]

y_val = train_labels[:100]
partial_y_train = train_labels[100:]

In [143]:
history = model.fit(x=partial_x_train, y=partial_y_train,validation_data=(x_val, y_val),epochs = 30)

Train on 528 samples, validate on 100 samples
Epoch 1/30


 32/528 [>.............................] - ETA: 8s

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  indices[0,2] = -3 is not in [0, 6000)
	 [[node sequential_15/embedding_8/embedding_lookup (defined at C:\Users\Windows 10\Anaconda3\envs\tf20\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]
	 [[Adam/Adam/update/AssignSubVariableOp/_75]]
  (1) Invalid argument:  indices[0,2] = -3 is not in [0, 6000)
	 [[node sequential_15/embedding_8/embedding_lookup (defined at C:\Users\Windows 10\Anaconda3\envs\tf20\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_distributed_function_32462]

Function call stack:
distributed_function -> distributed_function


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

In [ ]:
model.save('my_modelInter5')

In [ ]:
test_predictions = model.predict(train_examples)
test_predictions = np.argmax(test_predictions, axis=-1)

In [ ]:
result_df = intervention.copy()
result_df["Predictions"] = test_predictions
result_df["Predictions"] = result_df["Predictions"].replace([0,1,2],['N','P','Q'])

result_df.to_csv(
    "predictions.csv",
    columns=["Predictions","Blinding of intervention"],
    header=["Predictions","Blinding of intervention"])

In [ ]:
cm = tf.math.confusion_matrix(y_val, test_predictions[:100])
cm = cm/cm.numpy().sum(axis=1)[:, tf.newaxis]

In [ ]:
sns.heatmap(
    cm, annot=True)
plt.xlabel("Predicted")
plt.ylabel("True")